### 1. Importação das bibliotecas

In [2]:
# Manipulação dos dados
import numpy as np 
import pandas as pd 
import scipy as sp
import scipy.stats as st
# DataViz
import matplotlib.pyplot as plt
import seaborn as sns
#import seaborn.objects as so

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# Configuração de importação do dataset
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### 2. Importação do dataset e manipulação dos dados

In [ ]:
# Importação do dataset
df = pd.read_csv('/kaggle/input/churn-for-bank-customers/churn.csv')
# Mostrar primeiros registros
df.head()

In [ ]:
# Exclusão das colunas não utilizadas
df.drop(columns = {'RowNumber', 'CustomerId', 'Surname'}, inplace = True)
df.head()

In [ ]:
#Criar lista com nome das novas variáveis
novas_colunas = {
    'CreditScore': 'Score',
    'Geography': 'Pais',
    'Gender': 'Genero',
    'Age': 'Idade',
    'Tenure': 'Fidelidade',
    'Balance': 'Saldo',
    'NumOfProducts': 'QtdProdutos',
    'HasCrCard': 'TemCartao',
    'IsActiveMember': 'ClienteAtivo',
    'EstimatedSalary': 'Renda',
    'Exited': 'Churn'
    }
#renomeando as variáveis
df.rename(columns = novas_colunas, inplace = True)
#
del(novas_colunas)
#
df.head()

### 3. EDA - Análise Exploratória

In [ ]:
#Informações sobre missing values e tipo das variáveis
df.info()

In [ ]:
#Criar lista com o nome das colunas (variáveis)
variaveis = df.columns.to_list()
variaveis

In [ ]:
#Mostrar a quantidade de registros em cada variável
df[variaveis].nunique()

In [ ]:
# Listar variáveis quantitativa contínuas
var_quantitativas_continuas = [
    variavel for variavel in df.columns
        if df[variavel].dtype == 'float64' and variavel not in 'Churn']
var_quantitativas_continuas

In [ ]:
# Mostrar informaçõs estatísticas das variáveis quantitativas contínuas
df[var_quantitativas_continuas].describe().T

In [ ]:
# Listar variáveis quantitativa discretas
var_quantitativas_discretas = [
    variavel for variavel in df.columns
        if df[variavel].dtype == 'int64' and df[variavel].nunique() > 2 and variavel not in 'Churn']
var_quantitativas_discretas

In [ ]:
# Mostrar informaçõs estatísticas das variáveis quantitativas discretas
df[var_quantitativas_discretas].describe().T

In [ ]:
# Listar variáveis qualitativas nominais
var_qualitativas_nominais = [
    variavel for variavel in df.columns
        if df[variavel].dtype == 'object' and variavel not in 'Churn']
var_qualitativas_nominais

In [ ]:
# Mostrar informaçõs estatísticas das variáveis qualitativas nominais
df[var_qualitativas_nominais].describe().T

In [ ]:
#Alterando o conteúdo das variáveis categóricas
#* Localização
df['Pais'].value_counts()

In [ ]:
#Alterando os valores da variável #Localização
df['Pais'].replace({
    'France' : 'França',
    'Germany' : 'Alemanha',
    'Spain' : 'Espanha'
}, inplace = True)
#
df['Pais'].unique()

In [ ]:
#Alterando o conteúdo das variáveis categóricas
#* Gênero
df['Genero'].value_counts()

In [ ]:
#Alterando os valores da variável #Localização
#* Gênero
df['Genero'].replace({
    'Male' : 'Masculino',
    'Female' : 'Feminino'
}, inplace = True)
#
df['Genero'].unique()

In [ ]:
# Listar variáveis qualitativas ordinais
var_qualitativas_ordinais = [
    variavel for variavel in df.columns
        if df[variavel].dtype == 'int64' and df[variavel].nunique() == 2 and variavel not in 'Churn']
var_qualitativas_ordinais

#### 3.1. Análise univariada

In [ ]:
# Definição do Tema do Seaborn
sns.set_theme(
    context = 'paper', style = 'whitegrid', palette = 'muted',
    font = 'sans-serif', font_scale = 1, color_codes = True, 
    rc = None
)

##### 3.1.1. Score de Crédito

In [ ]:
#Criação dos subplots
fig, axs = plt.subplots(ncols = 1, nrows = 2, sharex=True, height_ratios=[0.8, 0.2])
#Título da imagem
plt.suptitle('Distribuição do Score de Crédito')
#Gráfico 1 - Histograma
sns.histplot(x = df['Score'], kde = True, ax = axs[0], stat = 'count', cumulative = False)
#Gráfico 2 - Boxplot
sns.boxplot(x = df['Score'], fliersize = 2, notch = True, ax = axs[1])